In [1]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",
    api_version="2024-05-01-preview",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [2]:
# Signal the thread to exit and wait for it to finish
from threading import Thread
from pyaudio import PyAudio, paInt16
from langchain_core.messages import HumanMessage, SystemMessage
from queue import Queue, Empty
import openai

openai.api_type = 'azure'

messages = [
    SystemMessage(content="You are a witty assistant that will use the chat history and the image provided by the user to answer its questions."),
    HumanMessage(content="hi!"),
]

def _tts(response, player):
    with openai.audio.speech.with_streaming_response.create(
        model="tts-hd",
        voice="nova",
        response_format="pcm",
        input=response,
    ) as stream:
        for chunk in stream.iter_bytes(chunk_size=1024):
            player.write(chunk)

def _tts_worker(queue, player):
    while True:
        try:
            response = queue.get(timeout=1)
            if response is None:
                break
            print(f'response: {response}')
            _tts(response, player)
        except Empty:
            continue

player = PyAudio().open(format=paInt16, channels=1, rate=24000, output=True)

queue = Queue()
tts_thread = Thread(target=_tts_worker, args=(queue, player))
tts_thread.start()

whole = ''
buffer = ''
buffer_size = 10  # Set your desired buffer size

for chunk in llm.stream(messages):
    buffer += chunk.content
    if len(buffer) >= buffer_size or chunk.content.strip() == '':
        if buffer.strip() != '':
            queue.put(f'buffer: {buffer}')
            whole += buffer
        buffer = ''

# Ensure any remaining content in the buffer is processed
if buffer.strip() != '':
    queue.put(buffer)
    whole += buffer

print(whole)

# Signal the thread to exit and wait for it to finish
queue.put(None)
tts_thread.join()

player.stop_stream()
player.close()

input("Press Enter to exit...")


response: Hello there
Hello there! Ready to dive into some fun? How can I assist you today?
response: ! Ready to
response:  dive into
response:  some fun?
response:  How can I
response:  assist you
response:  today?


''

In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(ai_msg)

'Ciao!'

In [5]:
chain = llm | parser
chain.invoke(messages)

'Ciao!'

In [6]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)
prompt_template

ChatPromptTemplate(input_variables=['language', 'text'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], template='Translate the following into {language}:')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))])

In [7]:
result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi')])

In [8]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi')]

In [9]:
chain.invoke(result)

'Ciao'

In [14]:
from langchain_openai import AzureChatOpenAI
from langchain.agents import load_tools, initialize_agent

import os
os.environ['SERPAPI_API_KEY'] = 'cee6a62a0f1dcab0791e15c3bc3e2642e7ab98fef43ed32b424d0047df22e748'

llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",
    api_version="2024-05-01-preview",
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


tools = load_tools(['serpapi', 'llm-math'], llm=llm)
agent = initialize_agent(tools, llm, agent='zero-shot-react-description', verbose=True)
agent.invoke('Who is the current leader of China? What is the largest prime number that is smmaller than his age?')
# agent.invoke('What is the largest prime number that is smmaller than 66?')



> Entering new AgentExecutor chain...
To answer the question, I first need to determine the current leader of China and his age. Then, I will find the largest prime number smaller than his age.

Action: Search
Action Input: "current leader of China 2023"
Observation: {'type': 'organic_result', 'title': 'Xi Jinping - Wikipedia', 'source': 'Wikipedia'}
Thought:I have identified that Xi Jinping is the current leader of China. Now, I need to find his age.

Action: Search
Action Input: "Xi Jinping age 2023"
Observation: About 70 years
Thought:Xi Jinping is approximately 70 years old in 2023. Now, I need to find the largest prime number that is smaller than 70.

Action: Calculator
Action Input: Find the largest prime number smaller than 70

ValueError: unknown format from LLM: The numexpr library is used for evaluating numerical expressions efficiently but does not directly provide functions for prime number calculations. However, I can still provide a Python expression to solve this problem using a different approach.

To find the largest prime number smaller than 70, one typically uses a prime-checking function within a loop to check numbers descending from 69 until a prime is found. This isn't directly executable using numexpr, but I'll show you a Python approach:

```python
def is_prime(n):
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

def largest_prime_below(n):
    for num in range(n-1, 1, -1):
        if is_prime(num):
            return num

largest_prime_below(70)
```

Running this code will give us the la